In [ ]:
import preamble

# Constants

In [ ]:
import numpy as np

# Experiment
USER_ID = 0

# Trajectory Planner
REFERENCE_TRAJECTORY_CSV_PATH = '../data/reference_path.csv'
INIT_POSE = np.array([-0.54, -0.27, 0.0, -1.21, -1.21, 1.25]) # Also used for Robot

# Transition Probabilities
state_map = {(0, 0): 0, (1, 0): 1, (0, 1): 2, (1, 1): 3}
A = np.array([
    [ 0.7180186,  0.6531676,  0.9194265, -1.1720577],
    [ 0.5751301,  0.7693612, -1.3269429,  0.6973043],
    [ 0.8416619, -1.0936067,  0.7530234,  0.6233168],
    [-2.9374210,  0.3163404,  0.3909466,  0.2452125]
])
B = np.array([
    [ 0.4821496, -0.3078547,  0.4216300, -1.8944792],
    [ 0.6335048,  0.3842422, -1.8434653,  0.4650253],
    [ 1.0996757, -1.8577393,  0.5245419,  0.3539045],
    [-2.3862197,  0.7957043, -0.5840249, -0.5731659],
])
C = np.array([
    [ 3.333601, -1.9594908, -3.6424782, -3.0082774],
    [-2.864783,  2.3264263, -3.7144790, -2.5864382],
    [-3.484102, -3.6938949,  3.1255682, -1.5355915],
    [-4.482538, -4.6248226, -2.1531882,  4.6947365],
])
prior_belief = np.full(4, 0.25)
U_p, U_v = 0, 0
P_hat, V_hat = 0.5, 0.5
dVdt = 0
DVDT_ENCOURAGEMENT_THRESHOLD = 1.0
BAYESIAN_FILTER_SAMPLING_RATE = 100

# Prior Distributions
# MEANS = np.array([
#     [-3.99849367, -3.16462103],
#     [-3.87261021, -3.14348198],
#     [-4.14477961, -3.16904538],
#     [-4.11880238, -3.24909682],
# ])
# COVARIANCES = np.array([
#     [[0.30300593, 0.04605989], [0.04605989, 0.55695469]],
#     [[0.30524614, 0.08586392], [0.08586392, 0.58394479]],
#     [[0.30183493, 0.0819836 ], [0.0819836 , 0.56152217]],
#     [[0.3647403 , 0.11946842], [0.11946842, 0.5560613 ]],
# ])

MEANS = np.array([
    [0.1, 0.05],
    [0.09, 0.04],
    [0.09, 0.04],
    [0.05, 0.025],
])

mean_cov = np.array([
    [0.31870683, 0.08334396],
    [0.08334396, 0.56462074]
])

COVARIANCES = np.array([mean_cov, mean_cov, mean_cov, mean_cov])

# Verbal PD
VERBAL_PD_SCALER = 4.0
K_v = 5.6 * VERBAL_PD_SCALER
B_v = 4.8 * VERBAL_PD_SCALER

# F2L
T_proj = 4
N_proj = 256
dt_proj = T_proj / N_proj

# Physical PD
PHYSICAL_PD_SCALER = 2
K_p = 19 * PHYSICAL_PD_SCALER
B_p = 11 * PHYSICAL_PD_SCALER

# Physical Admittance
M = 2
B = 17.7

# F2L
F2L_MODEL_PATH = '../data/F2L_models/svm_knn.pkl'
FINAL_IMPULSE_THRESHOLD = 0

# Real time data collection
TABLE_COLUMN_NAMES = [
    't',
    'x',
    'x_dot',
    'F_h',
    'ref_going_forward',
    'x_ref',
    'x_dot_ref',
    'e',
    'e_dot',
    'P_hat',
    'V_hat',
    'c_p',
    'c_v',
    'verbal_P_tracking_term',
    'verbal_D_tracking_term',
    'verbal_PD_tracking_term',
    'A_v',
    'U_v',
    'physical_P_tracking_term',
    'physical_D_tracking_term',
    'physical_PD_tracking_term',
    'A_p',
    'U_p',
    'successful_utterance',
    'is_speaking',
    'phrase',
    'U',
    'human_disturbance',
    'system_dynamics_input',
]
DATA_COLLECTION_SAMPLING_RATE = 200

# Real time data plotting
REAL_TIME_DATA_PLOTTING_ENABLED = True
PLOTTING_SAMPLING_RATE = 30

# Robot
ROBOT_IP = '169.254.9.43'
FORCE_EWMA_TAU = 0.5
TRANSLATIONAL_FORCE_DEADBAND = 2

# Experiment

This baseline is made up of all componentes: Physical PD, Physical Admittance, Verbal PD, Verbal Admittance, F2L, Compliance Estimator, and $J$ Optimization Method.

In [ ]:
from timer import Timer
from trajectory_planner import TrajectoryPlanner
from model import SVMKNNModel
import pickle
from vocalizer import Vocalizer
from virtual_dynamics import SimpleVirtualDynamics
from tabular_data_store import TabularDataStore
from plot_client import PlotClient
from robot import Robot
from console_command_thread import ConsoleCommandThread
import scipy.stats
from phrase_mapping import PHRASE_MAPPING, MEAN_WORDS_PER_PHRASE

#############
### SETUP ###
#############

### Experiment ###
experiment_timer = Timer()

### Trajectory Planner ###
traj = TrajectoryPlanner(REFERENCE_TRAJECTORY_CSV_PATH, INIT_POSE[:3])

### Compliance Estimator ###
compliance_estimator_timer = Timer()

### F2L ###
with open(F2L_MODEL_PATH, 'rb') as file:
    F2L_model: SVMKNNModel = pickle.load(file)
phrase = ''
voc = Vocalizer()
speaking_timer = Timer()

### System dynamics ###
vd = SimpleVirtualDynamics(M=M, B=B, K=0)

### Real time data collection ###
table = TabularDataStore(column_names=TABLE_COLUMN_NAMES)
table_timer = Timer()

### Real time data plotting ###
if REAL_TIME_DATA_PLOTTING_ENABLED:
    plot = PlotClient()

    plot.create_plot("force_plot", title="Force vs Time", xlabel="Time (s)", ylabel="Force")
    plot.create_line("force_plot", "Fx", color="r", label="Fx")
    plot.create_line("force_plot", "Fy", color="g", label="Fy")
    plot.create_line("force_plot", "Fz", color="b", label="Fz")

    plot.create_plot("velocity_plot", title="Velocity vs Time", xlabel="Time (s)", ylabel="Velocity")
    plot.create_line("velocity_plot", "vx", color="r", label="vx")
    plot.create_line("velocity_plot", "vy", color="g", label="vy")
    plot.create_line("velocity_plot", "vz", color="b", label="vz")

    plot.create_plot("error_plot", title="Error vs Time", xlabel="Time (s)", ylabel="Error")
    plot.create_line("error_plot", "ex", color="r", label="ex")
    plot.create_line("error_plot", "ey", color="g", label="ey")
    plot.create_line("error_plot", "ez", color="b", label="ez")

    plot.create_plot("compliance_plot", title="Estimated Compliance vs Time", xlabel="Time (s)", ylabel="Probability")
    plot.create_line("compliance_plot", "P_hat", color="orange", label="P̂ (Physical)")
    plot.create_line("compliance_plot", "V_hat", color="purple", label="V̂ (Verbal)")
    
    plot.create_plot("control_plot", title="Control Inputs vs Time", xlabel="Time (s)", ylabel="Control Norm")
    plot.create_line("control_plot", "U_p", color="cyan", label="‖U_p‖")
    plot.create_line("control_plot", "U_v", color="magenta", label="‖U_v‖")

    plot.create_plot("U_v_projection_plot", title="Projected U_v Profile", xlabel="Future Steps", ylabel="U_v")
    plot.create_line("U_v_projection_plot", "U_v_x", color="r", label="U_v,x")
    plot.create_line("U_v_projection_plot", "U_v_y", color="g", label="U_v,y")
    plot.create_line("U_v_projection_plot", "U_v_z", color="b", label="U_v,z")

    plot_timer = Timer()

z = None

### Robot ###
with Robot(
    ROBOT_IP,
    default_axes=Robot.TRANSLATION,
    init_pose=INIT_POSE,
    force_ewma_tau=FORCE_EWMA_TAU,
    translational_force_deadband=TRANSLATIONAL_FORCE_DEADBAND,
) as r, ConsoleCommandThread() as c:

    ##################
    ### EXPERIMENT ###
    ##################

    experiment_timer.reset()
    compliance_estimator_timer.reset()
    speaking_timer.reset()

    while c.is_alive():
        period_start = r.control.initPeriod()

        ### Current state ###
        t = experiment_timer.t()
        dt = experiment_timer.dt()
        x = r.get_pose()
        x_dot = r.get_velocity()
        F_h = r.get_force()

        ### Trajectory Planner ###
        ref_going_forward = traj.update_reference_trajectory(x)
        x_ref, x_dot_ref = traj.get_closest_target(x)

        ### Tracking errors ###
        e = x_ref - x
        e_dot = x_dot_ref - x_dot

        ### Compliance Estimator ###
        if compliance_estimator_timer.t() >= BAYESIAN_FILTER_SAMPLING_RATE**(-1):
            z = np.log(np.array([np.linalg.norm(e), np.linalg.norm(e_dot)]) + 1e-6)

            # Transition Update
            U_p_norm = np.linalg.norm(U_p)
            U_v_norm = int(voc.is_uttering())

            logits = A * U_p_norm + B * U_v_norm + C

            logits_stable = logits - np.max(logits, axis=1, keepdims=True)
            exp_logits = np.exp(logits_stable)
            T = exp_logits / exp_logits.sum(axis=1, keepdims=True)

            predict_belief = T.T @ prior_belief

            # Observation Update
            likelihoods = np.zeros(4)
            for i in range(4):
                likelihoods[i] = scipy.stats.multivariate_normal.pdf(z, mean=MEANS[i], cov=COVARIANCES[i])
            posterior_belief = predict_belief * likelihoods
            posterior_belief /= posterior_belief.sum()
            prior_belief = posterior_belief

            # Marginalize Joint State
            P_hat = posterior_belief[state_map[(1, 0)]] + posterior_belief[state_map[(1, 1)]]
            previous_V_hat = V_hat
            V_hat = posterior_belief[state_map[(0, 1)]] + posterior_belief[state_map[(1, 1)]]
            dVdt = (V_hat - previous_V_hat) / dt

            compliance_estimator_timer.reset()

        ### Guidance costs ###
        c_p = (P_hat + V_hat) / 2
        c_v = 1 - V_hat

        ### Verbal PD ###
        verbal_P_tracking_term = K_v * e
        verbal_D_tracking_term = B_v * e_dot
        verbal_PD_tracking_term = verbal_P_tracking_term + verbal_D_tracking_term

        ### Verbal Admittance ###
        A_v = c_p / (c_p + c_v)
        U_v = verbal_PD_tracking_term - A_v*F_h

        ### Physical PD ###
        physical_P_tracking_term = K_p * e
        physical_D_tracking_term = B_p * e_dot
        physical_PD_tracking_term = physical_P_tracking_term + physical_D_tracking_term

        ### Physical Admittance ###
        A_p = c_v / (c_p + c_v)
        U_p = physical_PD_tracking_term - A_p*F_h

        ### F2L ###
        x_sim = np.copy(x)
        x_dot_sim = np.copy(x_dot)

        U_v_profile = []

        for _ in range(N_proj):
            x_ref_proj, x_dot_ref_proj = traj.get_closest_target(x_sim)
            
            e_proj = x_ref_proj - x_sim
            e_dot_proj = x_dot_ref_proj - x_dot_sim

            U_v_proj = (K_v * e_proj + B_v * e_dot_proj) / (1 + A_v)
            U_v_profile.append(U_v_proj)

            U_p_proj = K_p * e_proj + B_p * e_dot_proj - A_p * U_v_proj

            U_proj = U_p_proj + U_v_proj

            a_proj = (U_proj - B * x_dot_sim) / M
            x_dot_sim += a_proj * dt_proj
            x_sim += x_dot_sim * dt_proj

        U_v_profile = np.array(U_v_profile)
        impulse_curve = U_v_profile.cumsum(axis=0) * dt_proj
        final_impuse = impulse_curve[-1, :]

        speaking_period = MEAN_WORDS_PER_PHRASE / (0.36 * 2.2 ** (2 - P_hat - V_hat))

        if not voc.is_uttering():
            if np.linalg.norm(final_impuse) < FINAL_IMPULSE_THRESHOLD or speaking_timer.t() <= speaking_period:
                phrase = ''
            else:
                words = F2L_model.force_to_phrase(impulse_curve[None, :])[0]
                phrase = PHRASE_MAPPING[' '.join(words).strip()]
                if dVdt > DVDT_ENCOURAGEMENT_THRESHOLD:
                    phrase = 'good job'
                speaking_timer.reset()

        successful_utterance = voc.utter(phrase)

        is_speaking = voc.is_uttering()

        ### System dynamics ###
        U = U_p + U_v
        human_disturbance = F_h - U_v
        system_dynamics_input = U + human_disturbance
        vd.apply_force(system_dynamics_input, dt)
        r.set_velocity(vd.get_velocity(), acceleration=10)

        ### Real time data collection ###
        if table_timer.t() >= DATA_COLLECTION_SAMPLING_RATE**(-1):
            table.append_row((
                t,
                x,
                x_dot,
                F_h,
                ref_going_forward,
                x_ref,
                x_dot_ref,
                e,
                e_dot,
                P_hat,
                V_hat,
                c_p,
                c_v,
                verbal_P_tracking_term,
                verbal_D_tracking_term,
                verbal_PD_tracking_term,
                A_v,
                U_v,
                physical_P_tracking_term,
                physical_D_tracking_term,
                physical_PD_tracking_term,
                A_p,
                U_p,
                successful_utterance,
                is_speaking,
                phrase,
                U,
                human_disturbance,
                system_dynamics_input,
            ))
            
            table_timer.reset()
            
        ### Real time data plotting ###
        if REAL_TIME_DATA_PLOTTING_ENABLED:
            if plot_timer.t() >= PLOTTING_SAMPLING_RATE**(-1):
                plot.update_line("force_plot", "Fx", (t, F_h[0]))
                plot.update_line("force_plot", "Fy", (t, F_h[1]))
                plot.update_line("force_plot", "Fz", (t, F_h[2]))

                plot.update_line("velocity_plot", "vx", (t, x_dot[0]))
                plot.update_line("velocity_plot", "vy", (t, x_dot[1]))
                plot.update_line("velocity_plot", "vz", (t, x_dot[2]))

                plot.update_line("error_plot", "ex", (t, z[0] if z is not None else 0))
                plot.update_line("error_plot", "ey", (t, z[1] if z is not None else 0))
                # plot.update_line("error_plot", "ez", (t, e[2]))

                plot.update_line("compliance_plot", "P_hat", (t, P_hat))
                plot.update_line("compliance_plot", "V_hat", (t, V_hat))

                plot.update_line("control_plot", "U_p", (t, np.linalg.norm(U_p)))
                plot.update_line("control_plot", "U_v", (t, np.linalg.norm(U_v)))

                timesteps = np.arange(len(U_v_profile)) * dt_proj

                plot.update_line("U_v_projection_plot", "proj_v_impulse_x", (timesteps, impulse_curve[:, 0]), mode="replace")
                plot.update_line("U_v_projection_plot", "proj_v_impulse_y", (timesteps, impulse_curve[:, 1]), mode="replace")
                plot.update_line("U_v_projection_plot", "proj_v_impulse_z", (timesteps, impulse_curve[:, 2]), mode="replace")

                plot_timer.reset()

        r.control.waitPeriod(period_start)

# table.to_pandas().to_pickle(f'../data/experiments/physical_only/{USER_ID}.pkl')